In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib as mplt
import json
from tbparse import SummaryReader

Helper to find the best run in a folder containing several runs

---

In [ ]:
def find_best_run(experiment_folder_list:list[str])->None:
    """
    Find best performing training run:

    parameters:
        experiment_folder_list:     List of folders containing different collections of experiments
    """

    best_mean_validation_KL = None

    for experiment_folder in experiment_folder_list:

        subfolders = os.listdir(experiment_folder)
        
        for subfolder in subfolders:

            full_path = os.path.join(experiment_folder,subfolder)
            
            reader_k = SummaryReader(full_path,extra_columns=set(["wall_time"]))
            df_k = reader_k.scalars
            df_red = df_k[(df_k["tag"] == "model_performance/mean_validation_KL")]
            kl_k = df_red["value"].values

            if (best_mean_validation_KL is None) or (best_mean_validation_KL > kl_k.min()):
                best_folder = full_path
                best_mean_validation_KL = kl_k.min()

            print(kl_k.min(),"\t",full_path)

    print("\n\nBest folder")
    print(best_folder)

search_best = False

if search_best:

    folder_list = [
        "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/"
        ]
    find_best_run(folder_list)

Set the paths

---

In [ ]:
EXPERIMENT_MODE = "SCALAR_THEORY_N16_TWO_BASE_PARAMETERS"

# Limits for plotting
lim_susceptibility = {
    "SCALAR_THEORY_N8_TWO_BASE_PARAMETERS":[-0.5,25],
    "SCALAR_THEORY_N16_TWO_BASE_PARAMETERS":[-0.5,40],
}

lim_binder = {
    "SCALAR_THEORY_N8_TWO_BASE_PARAMETERS":[-0.5,1],
    "SCALAR_THEORY_N16_TWO_BASE_PARAMETERS":[-1,1],
}

if EXPERIMENT_MODE == "SCALAR_THEORY_N8_TWO_BASE_PARAMETERS":

    model_to_eval = "best"
    N = 8

    path_dict = {
        "TRADE_no_grid":    "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0",
        "TRADE_grid":       "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0",
        "NLL_only":         "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0",
        "reverse_KL":       "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0",
        "reverse_KL_NLL":   "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0"
    }

    folder_dict_random_seeds = {
        "TRADE_no_grid":    "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/",
        "TRADE_grid":       "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/",
        "reverse_KL_NLL":   "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/"
    }

elif EXPERIMENT_MODE == "SCALAR_THEORY_N16_TWO_BASE_PARAMETERS":
    
    model_to_eval = "best"
    N = 16

    path_dict = {
        "NLL_only":         "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0",
        "reverse_KL":       "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0",
        "reverse_KL_NLL":   "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0",
        "TRADE_no_grid":    "../../results/runs_ScalarTheory/<Your experiment name>/lightning_logs/version_0"
    }

figure_folder = f"./Figures_{EXPERIMENT_MODE}/"

if not os.path.exists(figure_folder):
    os.makedirs(figure_folder)

In [ ]:
labels_dict = {
    "TRADE_no_grid":"TRADE (no grid)",
    "TRADE_grid":"TRADE (grid)",
    "NLL_only":"NLL only",
    "reverse_KL":"Reverse KLD",
    "reverse_KL_NLL":"Reverse KLD + NLL"
}

fs = 10

Create a table for the validation NLLs:

---

In [ ]:
def get_table_str_sub_table(
        rows_value_dict,
        rows_error_dict,
        row_key_to_row_label_dict,
        col_key_to_col_label_dict,
        rows_to_highlight = [],
        col_label_prefix = "NLL ",
        col_label_suffix = "$\downarrow$",
        highlight_color = "lightgray"):

    table_str = "\\begin{tabularx}{\\textwidth}{|c|"

    for i in range(len(col_key_to_col_label_dict.keys())):
        table_str = table_str + ">{\centering\\arraybackslash}X|"
    table_str = table_str+ "}\n\hline\n"

    # Column names
    for col_key in col_key_to_col_label_dict.keys():
        table_str += "&" + col_label_prefix + col_key_to_col_label_dict[col_key] + col_label_suffix
    table_str += "\\\\\n\hline\n"

    # For each column get the best performing entry
    is_best_dict = {}

    for row_label in rows_value_dict.keys():
        is_best_dict[row_label] = {}

        for col_label in col_key_to_col_label_dict.keys():
            is_best_dict[row_label][col_label] = False

    for col_label in col_key_to_col_label_dict.keys():

        best_row_i = None

        for row_label in rows_value_dict.keys():

            if (best_row_i is None) or (rows_value_dict[row_label][col_label] < best_row_val_i):
                best_row_i = row_label
                best_row_val_i = rows_value_dict[row_label][col_label]

        is_best_dict[best_row_i][col_label] = True

    # Fill rows
    for row_label in rows_value_dict.keys():
        
        # Highlight rows
        if row_label in rows_to_highlight:
            table_str += "\\rowcolor{" + highlight_color + "}"

        table_str += row_key_to_row_label_dict[row_label]
        
        for col_label in col_key_to_col_label_dict.keys():

            # Round the entries of the cell
            magnitude = np.floor(np.log10(abs(rows_error_dict[row_label][col_label]))) 
            magnitude = abs(int(magnitude - 2))

            if is_best_dict[row_label][col_label]:
                table_str += "&\\textbf{"+ f"{round(rows_value_dict[row_label][col_label],magnitude)}$\pm${round(rows_error_dict[row_label][col_label],magnitude)}"+"}"
            else:
                table_str += f"&{round(rows_value_dict[row_label][col_label],magnitude)}$\pm${round(rows_error_dict[row_label][col_label],magnitude)}"

        table_str += "\\\\\n"

    table_str += "\hline\n"

    table_str = table_str +"\end{tabularx}"

    return table_str

def get_table_str(
        rows_value_dict,
        rows_error_dict,
        row_key_to_row_label_dict,
        col_key_to_col_label_dict,
        rows_to_highlight = [],
        col_label_prefix = "NLL ",
        col_label_suffix = "$\downarrow$",
        highlight_color = "lightgray",
        values_per_row = 3):
    
    table_str = ""

    
    for i in range(0,len(col_key_to_col_label_dict.keys()),values_per_row):

        sub_table_column_names = list(col_key_to_col_label_dict.keys())[i:min(i + values_per_row,len(col_key_to_col_label_dict.keys()))]
        
        table_str += f"%Subtable {int(i / values_per_row) + 1}\n"

        sub_table_row_value_dict = {}
        sub_table_row_error_dict = {}
        sub_table_col_key_to_col_label_dict = {}

        for row_label in rows_value_dict.keys():

            sub_table_row_value_dict[row_label] = {}
            sub_table_row_error_dict[row_label] = {}

            for col_label in col_key_to_col_label_dict.keys():
                
                if col_label in sub_table_column_names:
                    sub_table_row_value_dict[row_label][col_label] = rows_value_dict.get(row_label).get(col_label)
                    sub_table_row_error_dict[row_label][col_label] = rows_error_dict.get(row_label).get(col_label)
                    sub_table_col_key_to_col_label_dict[col_label] = col_key_to_col_label_dict.get(col_label)

        sub_table_str = get_table_str_sub_table(
            rows_value_dict = sub_table_row_value_dict,
            rows_error_dict = sub_table_row_error_dict,
            row_key_to_row_label_dict = row_key_to_row_label_dict,
            col_key_to_col_label_dict = sub_table_col_key_to_col_label_dict,
            rows_to_highlight = rows_to_highlight,
            col_label_prefix = col_label_prefix,
            col_label_suffix = col_label_suffix,
            highlight_color = highlight_color
        )

        table_str += sub_table_str + "\n\n"

    return table_str

In [ ]:
nll_dict = {}
error_nll_dict = {}

col_key_to_label_dict = {}

for key in path_dict.keys():

    assert(f"Evaluation_{model_to_eval}" in os.listdir(path_dict[key]))

    with open(os.path.join(path_dict[key],f"Evaluation_{model_to_eval}","validation_nll_dict.json"),"r") as f:
        NLL_dict_key = json.load(f)
    f.close()

    nll_dict[key] = NLL_dict_key["nll"]
    error_nll_dict[key] = NLL_dict_key["error"]

    for col_key in nll_dict[key].keys():
        kappa_col = col_key.split('_')[0].split("=")[1]
        col_key_to_label_dict[col_key] = f"$\kappa = {kappa_col}$"

table_str_validation_nll = get_table_str(
    rows_to_highlight=["TRADE_no_grid","TRADE_grid"],
    rows_error_dict=error_nll_dict,
    rows_value_dict=nll_dict,
    row_key_to_row_label_dict=labels_dict,
    col_key_to_col_label_dict=col_key_to_label_dict
)

with open(os.path.join(figure_folder,f"{EXPERIMENT_MODE}_table_validation_NLL.txt"),"w") as f:
    f.write(table_str_validation_nll)
f.close()

Plot the effective sampling size:

---

In [ ]:
fig_ESS,ax_ESS = plt.subplots(1,1,figsize = (7,5))

colors = mplt.colormaps['Set1'].colors
counter = 0

for key in path_dict.keys():

    print(key)

    assert(f"Evaluation_{model_to_eval}" in os.listdir(path_dict[key]))

    # Load the spin distributions
    ESS_r_key = np.loadtxt(os.path.join(path_dict[key],f"Evaluation_{model_to_eval}","ESS_r.txt"))

    ax_ESS.errorbar(
        x = ESS_r_key[:,0],
        y = ESS_r_key[:,1],
        yerr = ESS_r_key[:,2],
        label = labels_dict[key],
        capsize = 3,
        marker = ".",
        ls = "dotted",
        color = colors[counter]
    )

    counter += 1

ax_ESS.hlines(y = 1.0,xmin = ESS_r_key[:,0].min() - 0.002,xmax = ESS_r_key[:,0].max() + 0.002,ls = "dotted",color = "k",label = "optimum")

ax_ESS.set_xlabel(r"$\kappa$",fontsize = fs)
ax_ESS.set_ylabel(r"ESS($\kappa$)",fontsize = fs)
ax_ESS.tick_params(axis='both', which='major', labelsize=fs)

plt.tight_layout()

handles, labels = [], []

for handle, label in zip(*ax_ESS.get_legend_handles_labels()):
    handles.append(handle)
    labels.append(label)

# Add a single legend below all subplots
fig_ESS.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=4,fontsize = fs)

plt.savefig(
        os.path.join(figure_folder,f"{EXPERIMENT_MODE}_ESS_r.pdf"),
        bbox_inches='tight'
)
plt.savefig(
        os.path.join(figure_folder,f"{EXPERIMENT_MODE}_ESS_r.jpeg"),
        bbox_inches='tight'
)
plt.close(fig_ESS)


Plot the physical observables

---

In [ ]:
# Load the data
data_dict = {}

for key in path_dict.keys():
        data_key = np.loadtxt(os.path.join(path_dict[key],f"Evaluation_{model_to_eval}","summary_phyiscs_properties_lambda_0.02.txt"),skiprows=1)
        data_dict[key] = data_key


property_cols = {
        "action":3,
        "magnetization":1,
        "binder_cumulant":7,
        "susceptibility":5,
}

reference_cols = {
        "action":3,
        "magnetization":1,
        "binder_cumulant":5,
        "susceptibility":7,
}

y_labels = {
        "action":r"$\left<s(\kappa)\right>$",
        "magnetization":r"$\left<|m(\kappa)|\right>$",
        "binder_cumulant":r"$U_L(\kappa)$",
        "susceptibility":r"$\chi^2$",
}

# Get the reference simulation
reference = np.loadtxt(f"../../data/ScalarTheory/validation_data/N_{N}_LANGEVIN_SPECIFIC/summary_lambda_0.02_0.txt",skiprows = 1)

steps_size = 4

for property in property_cols.keys():

        fig_i,ax_i = plt.subplots(1,1,figsize = (7,3.0))

        colors = mplt.colormaps['Set1'].colors
        counter = 0

        ax_i.set_xlabel(r"$\kappa$",fontsize = fs)
        ax_i.set_ylabel(y_labels[property],fontsize = fs)
        ax_i.tick_params(axis='both', which='major', labelsize=fs)

        mask = (reference[:,0] <=  data_dict[key][:,0].max())

        ax_i.errorbar(
                x = reference[:,0][mask],
                y = reference[:,reference_cols[property]][mask],
                yerr = reference[:,reference_cols[property]+1][mask],
                label = "MCMC",
                capsize = 3,
                marker = ".",
                ls = "dotted",
                color = "k"
        )

        for key in path_dict.keys():
                ax_i.errorbar(
                        x = data_dict[key][:,0][::steps_size],
                        y = data_dict[key][:,property_cols[property]][::steps_size],
                        yerr = data_dict[key][:,property_cols[property]+1][::steps_size],
                        label = labels_dict[key],
                        capsize = 3,
                        marker = ".",
                        ls = "dotted",
                        color = colors[counter]
                )

                counter += 1
        if property == "susceptibility":
                ax_i.set_ylim([lim_susceptibility[EXPERIMENT_MODE][0],lim_susceptibility[EXPERIMENT_MODE][1]])

        if property == "binder_cumulant":
                ax_i.set_ylim([lim_binder[EXPERIMENT_MODE][0],lim_binder[EXPERIMENT_MODE][1]])
        
        ax_i.set_xlim([data_dict[key][:,0].min() - 0.005,data_dict[key][:,0].max() + 0.005])

        plt.tight_layout()

        handles, labels = [], []

        for handle, label in zip(*ax_i.get_legend_handles_labels()):
                handles.append(handle)
                labels.append(label)

        # Add a single legend below all subplots
        fig_i.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.05), ncol=3,fontsize = fs)

        plt.savefig(
                os.path.join(figure_folder,f"{EXPERIMENT_MODE}_{property}.pdf"),
                bbox_inches='tight'
        )
        plt.savefig(
                os.path.join(figure_folder,f"{EXPERIMENT_MODE}_{property}.jpeg"),
                bbox_inches='tight'
        )
        plt.close(fig_i)  

Plot the spin distributions of the INN

---

In [ ]:
fig,axes = plt.subplots(3,2,figsize = (13,12))

axes_flat = axes.reshape(-1)

fs = 20

for q,key in enumerate(path_dict.keys()):

    print(key)
     
    assert(f"Evaluation_{model_to_eval}" in os.listdir(path_dict[key]))

    if key == "TRADE_no_grid":

        #Get the ground truth spins_validation_data.txt
        spins_target = np.loadtxt(os.path.join(path_dict[key],f"Evaluation_{model_to_eval}","spins_validation_data.txt"))

        kappas_list = [float(a) for a in header.split('\t')[1:]]

        cmap = get_cmap('viridis')
        colors = [cmap(i / len(kappas_list)) for i in range(len(kappas_list))]

        for i in range(1,spins_target.shape[-1]):
            axes_flat[0].stairs(edges = spins_target[:,0],values=spins_target[:-1,i],color = colors[i-1],label = r"$\kappa = $"+f"{kappas_list[i-1]}",lw = 2)

        axes_flat[0].set_xlabel(r"$\phi(x)$",fontsize = fs)
        axes_flat[0].set_ylabel(r"$p(\phi(x)$)",fontsize = fs)
        axes_flat[0].tick_params(axis='both', which='major', labelsize=fs)
        axes_flat[0].set_title("validation data",fontsize = fs)

    
    #Load the spin distributions
    spins_key = np.loadtxt(os.path.join(path_dict[key],f"Evaluation_{model_to_eval}","spins_INN_data.txt"))

    with open(os.path.join(path_dict[key],f"Evaluation_{model_to_eval}","spins_INN_data.txt"),"r") as f:
        header = f.readline()
    f.close()

    kappas_list = [float(a) for a in header.split('\t')[1:]]

    cmap = get_cmap('viridis')
    colors = [cmap(i / len(kappas_list)) for i in range(len(kappas_list))]

    for i in range(1,spins_key.shape[-1]):
        axes_flat[q+1].stairs(edges = spins_key[:,0],values=spins_key[:-1,i],color = colors[i-1],label = r"$\kappa = $"+f"{kappas_list[i-1]}",lw = 2)

    axes_flat[q+1].set_xlabel(r"$\phi(x)$",fontsize = fs)
    axes_flat[q+1].set_ylabel(r"$p(\phi(x)$)",fontsize = fs)
    axes_flat[q+1].tick_params(axis='both', which='major', labelsize=fs)
    axes_flat[q+1].set_title(labels_dict[key],fontsize = fs)

    plt.tight_layout()

handles, labels = [], []

for handle, label in zip(*axes_flat[0].get_legend_handles_labels()):
        handles.append(handle)
        labels.append(label)

if N == 16:
    axes_flat[-1].get_xaxis().set_ticks([])
    axes_flat[-1].get_yaxis().set_ticks([])
    axes_flat[-1].axis("off")
# Add a single legend below all subplots
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5,-0.01), ncol=5,fontsize = fs)


plt.savefig(
    os.path.join(figure_folder,f"{EXPERIMENT_MODE}_spin_distributions_combined.pdf"),
    bbox_inches='tight'
)
plt.close(fig_i)

Evaluate the performance for different random seeds:

---

In [ ]:
# ESS
fig_ESS,ax_ESS = plt.subplots(3,1,figsize = (13,5 * 3))

colors = mplt.colormaps['Set1'].colors
counter = 0
fs = 20
min_val = 1.0

for counter,key in enumerate( folder_dict_random_seeds.keys()):

    Ess_tensor_key = None

    # Load all the experiments in the folder
    for subfolder in os.listdir(folder_dict_random_seeds[key]):

        assert(f"Evaluation_{model_to_eval}" in os.listdir(os.path.join(folder_dict_random_seeds[key],subfolder)))

        ESS_r_key_i = np.loadtxt(os.path.join(folder_dict_random_seeds[key],subfolder,f"Evaluation_{model_to_eval}","ESS_r.txt"))

        if Ess_tensor_key is None:
            Ess_tensor_key = ESS_r_key_i[:,1].reshape(-1,1)

        else:
            Ess_tensor_key = np.concatenate((Ess_tensor_key,ESS_r_key_i[:,1].reshape(-1,1)),1)

    means_i= Ess_tensor_key.mean(-1)
    std_i = Ess_tensor_key.std(-1)

    ax_ESS[counter].plot(ESS_r_key_i[:,0],means_i,label = r"$\left<ESS\right>$",color = colors[counter],lw = 3)
    ax_ESS[counter].fill_between(ESS_r_key_i[:,0],means_i - std_i,means_i + std_i,color = colors[counter],alpha = 0.3,label = r"$\left<ESS\right>\pm std(ESS)$")

    ax_ESS[counter].hlines(y = 1.0,xmin = ESS_r_key_i[:,0].min() - 0.002,xmax = ESS_r_key_i[:,0].max() + 0.002,ls = "dotted",color = "k",label = "optimum",lw = 3)

    ax_ESS[counter].set_xlabel(r"$\kappa$",fontsize = fs)
    ax_ESS[counter].set_ylabel(r"ESS($\kappa$)",fontsize = fs)
    ax_ESS[counter].tick_params(axis='both', which='major', labelsize=fs)

    if min_val > np.min(means_i - std_i):
        min_val = np.min(means_i - std_i)

    ax_ESS[counter].set_title(labels_dict[key],fontsize = fs)

    ax_ESS[counter].legend(fontsize = fs)

    counter += 1

for counter,key in enumerate( folder_dict_random_seeds.keys()):
    ax_ESS[counter].set_ylim([min_val - 0.01,1.1])

plt.tight_layout()

plt.savefig(
        os.path.join(figure_folder,f"{EXPERIMENT_MODE}_ESS_r_random_seeds.pdf"),
        bbox_inches='tight'
)
plt.close(fig_ESS)